In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

## READING FILES

In [2]:
date_respuesta = '2020-10-12'
date_respuesta = pd.to_datetime(date_respuesta)

In [3]:
# disp = pd.read_excel(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2020\10. OCTUBRE\Dispersión-20201012.xlsx'
#                      ,sheet_name='Swap del día',converters={'CLABE':str,'report_id':str})
disp = pd.read_csv(r'C:\Users\Cristian.Aparicio\Google Drive\Settlement\5.-HISTORICO FILES PAGOS\2020\10. OCTOBER\SWAP 2020-10-12.csv'
                     ,dtype={'bank_account_number':object,'report_id':object})
rs1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\2020\10. OCTUBRE\Respuesta-Swap 2020-10-12.csv',skiprows=11
                     ,dtype={'Descripcion':object,'Referencia Numerica':object,'Rastreo CEP':object})
ae1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Google Drive\Settlement\5.-HISTORICO FILES PAGOS\2020\10. OCTOBER\SwapDOWNLOAD 2020-10-12.csv'
                  ,names=['Nombre','Blank','amount','CLABE','report_id'],dtype={'CLABE':object,'report_id':object},skiprows=[0])
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                   ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})


# BIN

In [4]:
bin1.head()

,BIN,Clave de la institución,Nombre de la institución
0,138,40138,ABC CAPITAL
1,102,40102,ACCENDO BANCO
2,133,40133,ACTINVER
3,062,40062,AFIRME
4,638,90638,AKALA


In [5]:
bin1.shape

(87, 3)

# ARCHIVO ENVIADO A SWAP

In [6]:
ae1.head()

,Nombre,Blank,amount,CLABE,report_id
0,Luis Alberto Hernandez Infante,nan,"1,006.15",012496015159949311,CLIP-GRKWPXGDN
1,Virginia Alejandra Barradas Prez,nan,122.65,137463103868892825,CLIP-P585CLCTJ
2,MARIA CRISTINA PEREZ X,nan,"1,375.08",137190103732530285,CLIP-MVNF37SXM
3,David Nolasco Resendiz,nan,301.85,127180013090386618,CLIP-T5WH6F6D8
4,JORGE MIGUEL ARIAS ARGUELLES,nan,"1,245.71",012910004651609361,CLIP-5H424LJ8R


In [7]:
ae1.shape

(34268, 5)

In [8]:
ae1['report_id'] = ae1['report_id'].str[5:].replace(' ', '',regex=True)
ae1['BIN'] = ae1['CLABE'].str[:3].replace(' ', '',regex=True)

In [9]:
ae_final = ae1.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
ae_final.head()

,Nombre,Blank,amount,CLABE,report_id,BIN,Clave de la institución,Nombre de la institución
0,Luis Alberto Hernandez Infante,nan,"1,006.15",012496015159949311,GRKWPXGDN,012,40012,BBVA BANCOMER
1,Virginia Alejandra Barradas Prez,nan,122.65,137463103868892825,P585CLCTJ,137,40137,BANCOPPEL
2,MARIA CRISTINA PEREZ X,nan,"1,375.08",137190103732530285,MVNF37SXM,137,40137,BANCOPPEL
3,David Nolasco Resendiz,nan,301.85,127180013090386618,T5WH6F6D8,127,40127,AZTECA
4,JORGE MIGUEL ARIAS ARGUELLES,nan,"1,245.71",012910004651609361,5H424LJ8R,012,40012,BBVA BANCOMER


In [10]:
ae_final.shape

(34268, 8)

# ARCHIVO RESPUESTA

In [11]:
rs1['Descripcion'] = rs1['Descripcion'].str.strip()

In [12]:
rs1['Date_respuesta'] = date_respuesta
rs1['Metodo de dispersion'] = 'SWAP'
rs1.rename(columns ={'Descripcion':'report_id'}, inplace=True)
rs1['Referencia Numerica'].fillna('SIN REFERENCIA',inplace=True)
rs1['Rastreo CEP'].fillna('SIN CEP',inplace=True)
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion
0,1100001,SIN CEP,LUISA FERNANDA CALZADA MARTINEZ,S6YGLV29Z,'137180104117328063,BANCOPPEL,14.00,2.15,paid,NaN,2020-10-12,SWAP
1,1100002,SIN CEP,BRENDA ITZEL MENDOZA URBANO,5Y9B23K8Q,'002180903507386213,BANAMEX,"1,800.48",2.15,paid,NaN,2020-10-12,SWAP
2,1100003,SIN CEP,MARIANA SANCHEZ VERASTEGUI,V3VGFH8T3,'012180029543501625,BBVA BANCOMER,86.24,2.15,paid,NaN,2020-10-12,SWAP
3,1100004,SIN CEP,GEORGINA ABASCAL GARCIA,HYDQGYVJS,'012650026165992815,BBVA BANCOMER,0.95,2.15,paid,NaN,2020-10-12,SWAP
4,1100005,SIN CEP,MALBER LILI PUERTO VAZQUEZ,45NJHW5DL,'012180015153133109,BBVA BANCOMER,65.40,2.15,paid,NaN,2020-10-12,SWAP


In [13]:
rs1.shape

(34268, 12)

In [14]:
rs1['Rastreo CEP'].str[:4].unique()

array(['SIN ', 'CLIP'], dtype=object)

In [15]:
rs1.loc[rs1['Rastreo CEP'].str.contains("'085"), "Linea_rastreo"] = rs1['Rastreo CEP'].str[1:]
rs1.loc[~rs1['Rastreo CEP'].str.contains("'085"), "Linea_rastreo"] = rs1['Rastreo CEP']


In [16]:
rs1.loc[rs1['Rastreo CEP'].str[:4] == "'085", "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == 'CLIP', "Banco_emisor"] = 'STP'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '2020', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '#N/D', "Banco_emisor"] = 'SANTANDER'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '1767', "Banco_emisor"] = 'BANAMEX'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '0', "Banco_emisor"] = 'CEP PENDIENTE DE ENVIO'
rs1.loc[rs1['Rastreo CEP'].str[:4] == '#¡REF!', "Banco_emisor"] = 'CEP PENDIENTE DE ENVIO'


In [17]:
rs1.loc[rs1['Banco_emisor'] == 'BANAMEX', "Emisor"] = '40002'
rs1.loc[rs1['Banco_emisor'] == 'STP', "Emisor"] = '90646'
rs1.loc[rs1['Banco_emisor'] == 'SANTANDER', "Emisor"] = '40014' 


In [18]:
rs1['Cuenta'].str[0:1].unique()

array(["'"], dtype=object)

In [19]:
rs1['BIN'] = rs1['Cuenta'].str[1:4]
rs1['CLABE'] = rs1['Cuenta'].str[1:]


In [20]:
rs1.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE
0,1100001,SIN CEP,LUISA FERNANDA CALZADA MARTINEZ,S6YGLV29Z,'137180104117328063,BANCOPPEL,14.00,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,137,137180104117328063
1,1100002,SIN CEP,BRENDA ITZEL MENDOZA URBANO,5Y9B23K8Q,'002180903507386213,BANAMEX,"1,800.48",2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,002,002180903507386213
2,1100003,SIN CEP,MARIANA SANCHEZ VERASTEGUI,V3VGFH8T3,'012180029543501625,BBVA BANCOMER,86.24,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,012,012180029543501625
3,1100004,SIN CEP,GEORGINA ABASCAL GARCIA,HYDQGYVJS,'012650026165992815,BBVA BANCOMER,0.95,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,012,012650026165992815
4,1100005,SIN CEP,MALBER LILI PUERTO VAZQUEZ,45NJHW5DL,'012180015153133109,BBVA BANCOMER,65.40,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,012,012180015153133109


In [21]:
rs1.shape

(34268, 17)

In [22]:
rs2 = rs1.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
rs2['LINEA_CEP'] = rs2['Date_respuesta'].astype(str).replace(' ', '',regex=True)+','+rs2['Linea_rastreo'].astype(str)+','+rs2['Emisor'].astype(str).replace(' ', '',regex=True)+','+rs2['Clave de la institución'].astype(str).replace(' ', '',regex=True)+','+rs2['CLABE'].astype(str).replace(' ', '',regex=True)+','+rs2['Importe'].astype(str).replace(' ', '',regex=True)
rs2.head()

,Referencia Numerica,Rastreo CEP,Nombre,report_id,Cuenta,Banco,Importe,Comision,Estatus,Mensaje,Date_respuesta,Metodo de dispersion,Linea_rastreo,Banco_emisor,Emisor,BIN,CLABE,Clave de la institución,Nombre de la institución,LINEA_CEP
0,1100001,SIN CEP,LUISA FERNANDA CALZADA MARTINEZ,S6YGLV29Z,'137180104117328063,BANCOPPEL,14.00,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,137,137180104117328063,40137,BANCOPPEL,"2020-10-12,SIN CEP,nan,40137,13718010411732806..."
1,1100002,SIN CEP,BRENDA ITZEL MENDOZA URBANO,5Y9B23K8Q,'002180903507386213,BANAMEX,"1,800.48",2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,002,002180903507386213,40002,BANAMEX,"2020-10-12,SIN CEP,nan,40002,00218090350738621..."
2,1100003,SIN CEP,MARIANA SANCHEZ VERASTEGUI,V3VGFH8T3,'012180029543501625,BBVA BANCOMER,86.24,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,012,012180029543501625,40012,BBVA BANCOMER,"2020-10-12,SIN CEP,nan,40012,01218002954350162..."
3,1100004,SIN CEP,GEORGINA ABASCAL GARCIA,HYDQGYVJS,'012650026165992815,BBVA BANCOMER,0.95,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,012,012650026165992815,40012,BBVA BANCOMER,"2020-10-12,SIN CEP,nan,40012,01265002616599281..."
4,1100005,SIN CEP,MALBER LILI PUERTO VAZQUEZ,45NJHW5DL,'012180015153133109,BBVA BANCOMER,65.40,2.15,paid,NaN,2020-10-12,SWAP,SIN CEP,NaN,NaN,012,012180015153133109,40012,BBVA BANCOMER,"2020-10-12,SIN CEP,nan,40012,01218001515313310..."


In [23]:
rs2.shape

(34268, 20)

# ARCHIVO DISPERSION

In [24]:
disp['report_id'] = disp['report_id'].str.strip()
disp['report_date_local'] = pd.to_datetime(disp['report_date_local'])

In [25]:
disp.loc[disp['report_date_local'] == date_respuesta, 'tipo'] = 'Pago del día'
disp.loc[disp['report_date_local'] != date_respuesta, 'tipo'] = 'Reprogramación'
disp.drop(['bank_name','bank_account_number','status','swap','payment_date_local'],axis=1,inplace=True)
disp.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,swap
0,0001ded3-6a7a-434b-bf23-8379b7ac89e1,Pelicanos,doriandahr@gmail.com,GRKWPXGDN,2020-10-12,"1,006.15",Pago del día,Luis Alberto Hernandez Infante
1,00023bd5-6149-4ef9-95be-edb61d22588d,LAS PEKES,alebarradas1@hotmail.com,P585CLCTJ,2020-10-12,122.65,Pago del día,Virginia Alejandra Barradas Prez
2,0003f6d6-1ced-46b6-b25b-c3ce3f289214,Cellairis Espacio Ags,marcosrios1971@icloud.com,MVNF37SXM,2020-10-12,"1,375.08",Pago del día,MARIA CRISTINA PEREZ X
3,0005b69f-11e4-405f-8a59-6ed9b8c42752,Inked Scars,davidpunkrocklove@gmail.com,T5WH6F6D8,2020-10-12,301.85,Pago del día,David Nolasco Resendiz
4,0008492a-25f3-464c-bc88-9aa81977536e,jetcar,jomiar18@hotmail.com,5H424LJ8R,2020-10-12,"1,245.71",Pago del día,JORGE MIGUEL ARIAS ARGUELLES


In [26]:
disp_2 = disp.merge(rs2[['report_id','Estatus','Nombre de la institución','CLABE','Banco_emisor','LINEA_CEP','Referencia Numerica']] ,on='report_id', how='left')
disp_2.sort_values(by=['Estatus'],ascending=True,inplace=True)
disp_2.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,swap,Estatus,Nombre de la institución,CLABE,Banco_emisor,LINEA_CEP,Referencia Numerica
14208,6b859539-8c14-4339-aca4-fc078d95ea37,Transportes,aruby6699@gmail.com,BDWBHNKSW,2020-10-12,"2,203.95",Pago del día,Dayana Citlally Villalobos Padilla,failed,BBVA BANCOMER,012180015137669233,NaN,"2020-10-12,SIN CEP,nan,40012,01218001513766923...",1100592
34174,ff31dba9-79c8-4524-9b08-48c6564a8b7c,El pachangon y clown town,giomdz20@gmail.com,SMX8DVYMG,2020-10-12,253.93,Pago del día,Geovanna mendez hernandez,failed,BBVA BANCOMER,012180015804627809,NaN,"2020-10-12,SIN CEP,nan,40012,01218001580462780...",1232790
10271,4de004c3-e888-416f-86d4-fc5c21974359,El Patio,alexandroalmendarez@gmail.com,5FHWMH825,2020-10-12,"2,919.73",Pago del día,Bryan Alexandro Almendarez Valles,failed,BBVA BANCOMER,012180015514148445,NaN,"2020-10-12,SIN CEP,nan,40012,01218001551414844...",1209838
7950,3c9c933b-34a8-41db-8e16-6edecbcb9229,Ventas de garage,botafoj@gmail.com,S8HYJ29MW,2020-10-12,"3,788.15",Pago del día,AndrÃ©s GonzÃ¡lez Acevedo,failed,SANTANDER,014420200126818827,NaN,"2020-10-12,SIN CEP,nan,40014,01442020012681882...",1100330
22835,abb3bd6c-b7de-4504-928d-51ac8834efa6,Nails Shop Areli Carvajal,areligabo@gmail.com,2MRQB488V,2020-10-12,536.61,Pago del día,Areli Carvajal,failed,BBVA BANCOMER,012872015106684971,NaN,"2020-10-12,SIN CEP,nan,40012,01287201510668497...",1221890


In [27]:
disp_2['LINEA_CEP'] = np.where(disp_2['LINEA_CEP'].str.contains('SIN CEP'), disp_2['Referencia Numerica'], disp_2['LINEA_CEP'])
disp_2.loc[disp_2['Banco_emisor'] != 'STP', 'Banco_emisor'] = 'SANTANDER - REFERENCIA NUMERICA'
disp_2.loc[disp_2['Nombre de la institución'] == 'SANTANDER', 'LINEA_CEP'] = 'COMPROBANTE MISMO BANCO SANTANDER'
disp_2.loc[disp_2['LINEA_CEP'] == 'COMPROBANTE MISMO BANCO SANTANDER', 'Banco_emisor'] = 'SANTANDER'
disp_2.loc[disp_2['Estatus'] == 'failed', 'LINEA_CEP'] = ''
disp_2.loc[disp_2['Estatus'] == 'failed', 'Banco_emisor'] = ''

In [28]:
for col in disp_2.columns: 
    print(col) 

merchant_id
merchant_name
user_email
report_id
report_date_local
total_due
tipo
swap
Estatus
Nombre de la institución
CLABE
Banco_emisor
LINEA_CEP
Referencia Numerica


In [29]:
disp_final = disp_2[['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','tipo'
                    ,'Nombre de la institución','bank_account_owner_name','CLABE','Estatus','LINEA_CEP','Banco_emisor']]
disp_final.rename(columns ={'Nombre de la institución':'bank','Estatus':'status','amount':'total_due'}, inplace=True)
disp_final.sort_values(by=['status'], ascending=True, inplace=True)
disp_final.head()

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,swap,CLABE,status,LINEA_CEP,Banco_emisor
14208,6b859539-8c14-4339-aca4-fc078d95ea37,Transportes,aruby6699@gmail.com,BDWBHNKSW,2020-10-12,"2,203.95",Pago del día,BBVA BANCOMER,Dayana Citlally Villalobos Padilla,012180015137669233,failed,,
24211,b5d448bb-194d-47fc-bc01-5f0f0d41efbc,Feel Good Inc.,p_prune@hotmail.com,FG6WDLJY3,2020-10-12,252.98,Pago del día,SCOTIABANK,Virginia Valle Rodriguez,044180256027312896,failed,,
21575,a26c77b4-ca1e-4a28-a01c-082ca15b8c20,Cherry Hot,facerscar61@gmail.com,CS4XG7PSV,2020-10-12,95.82,Pago del día,BANJERCITO,Elder David Martnez Calvo,019972000162698848,failed,,
31973,ee8d09c1-94ed-4050-a47d-3046433423e5,Restaurante De Mariscos El Paraiso,eduardo93zt@gmail.com,RRFDDK94T,2020-10-02,301.85,Reprogramación,BBVA BANCOMER,Jose Eduardo Zapo Toga,012180015151595192,failed,,
31974,ee8d09c1-94ed-4050-a47d-3046433423e5,Restaurante De Mariscos El Paraiso,eduardo93zt@gmail.com,PTD5FDJLL,2020-10-09,721.55,Reprogramación,BBVA BANCOMER,Jose Eduardo Zapo Toga,012180015151595192,failed,,


In [30]:
disp_final.shape

(34268, 13)

# ERROR

In [31]:
error = disp_final[(disp_final['status'] == 'failed')]
error.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,swap,CLABE,status,LINEA_CEP,Banco_emisor
14208,6b859539-8c14-4339-aca4-fc078d95ea37,Transportes,aruby6699@gmail.com,BDWBHNKSW,2020-10-12,"2,203.95",Pago del día,BBVA BANCOMER,Dayana Citlally Villalobos Padilla,012180015137669233,failed,,
24211,b5d448bb-194d-47fc-bc01-5f0f0d41efbc,Feel Good Inc.,p_prune@hotmail.com,FG6WDLJY3,2020-10-12,252.98,Pago del día,SCOTIABANK,Virginia Valle Rodriguez,044180256027312896,failed,,
21575,a26c77b4-ca1e-4a28-a01c-082ca15b8c20,Cherry Hot,facerscar61@gmail.com,CS4XG7PSV,2020-10-12,95.82,Pago del día,BANJERCITO,Elder David Martnez Calvo,019972000162698848,failed,,
31973,ee8d09c1-94ed-4050-a47d-3046433423e5,Restaurante De Mariscos El Paraiso,eduardo93zt@gmail.com,RRFDDK94T,2020-10-02,301.85,Reprogramación,BBVA BANCOMER,Jose Eduardo Zapo Toga,012180015151595192,failed,,
31974,ee8d09c1-94ed-4050-a47d-3046433423e5,Restaurante De Mariscos El Paraiso,eduardo93zt@gmail.com,PTD5FDJLL,2020-10-09,721.55,Reprogramación,BBVA BANCOMER,Jose Eduardo Zapo Toga,012180015151595192,failed,,


In [32]:
error.shape

(64, 13)

In [33]:
error.drop(['bank_account_owner_name','CLABE','status','LINEA_CEP','Banco_emisor'],axis=1,inplace=True)

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [34]:
error.loc[error['bank'] == 'SANTANDER', 'Check'] = 'SPEI DEVUELTO MISMO BANCO'
error.loc[error['bank'] != 'SANTANDER', 'Check'] = 'SPEI DEVUELTO'

C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Cristian.Aparicio\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [35]:
error_final = error.merge(rs2[['report_id','Mensaje']] ,on='report_id', how='left')
error_final['Mensaje'].fillna('NOT FOUND',inplace=True)
error_final.head()


,merchant_id,merchant_name,user_email,report_id,report_date_local,total_due,tipo,bank,Check,Mensaje
0,6b859539-8c14-4339-aca4-fc078d95ea37,Transportes,aruby6699@gmail.com,BDWBHNKSW,2020-10-12,"2,203.95",Pago del día,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...
1,b5d448bb-194d-47fc-bc01-5f0f0d41efbc,Feel Good Inc.,p_prune@hotmail.com,FG6WDLJY3,2020-10-12,252.98,Pago del día,SCOTIABANK,SPEI DEVUELTO,Cuenta inexistente
2,a26c77b4-ca1e-4a28-a01c-082ca15b8c20,Cherry Hot,facerscar61@gmail.com,CS4XG7PSV,2020-10-12,95.82,Pago del día,BANJERCITO,SPEI DEVUELTO,Falta informacion mandatoria para completar el...
3,ee8d09c1-94ed-4050-a47d-3046433423e5,Restaurante De Mariscos El Paraiso,eduardo93zt@gmail.com,RRFDDK94T,2020-10-02,301.85,Reprogramación,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...
4,ee8d09c1-94ed-4050-a47d-3046433423e5,Restaurante De Mariscos El Paraiso,eduardo93zt@gmail.com,PTD5FDJLL,2020-10-09,721.55,Reprogramación,BBVA BANCOMER,SPEI DEVUELTO,Excede el limite de saldo autorizado de la cue...


In [36]:
error_final.shape

(64, 10)

# ARCHIVOS FINALES

### ARCHIVO DISPERSION

In [37]:
writer = pd.ExcelWriter('DISPERSION PREVIA SWAP.xlsx', engine='xlsxwriter', options={'encoding':'utf-8'})
disp_final.to_excel(writer, sheet_name='SWAP DEL DIA',index=False)
error_final.to_excel(writer, sheet_name='ERROR',index=False)
writer.save()